---
Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import tensorflow
from tensorflow import keras as kr

---
# Función de activación custom

Crear una función de activación custom es sencillo, tan solo hay que crear una función como haríamos siempre.  
Una vez definida esta función correctamente solo tendremos que introducirla en "activation", en vez de las que vienen por defecto (relu, sigmoid, etc...) a la hora de definir el modelo.


In [ ]:
# Función de activación customizada que devuelve la media entre la relu y la sigmoid
def custom_activation1(input):
    return (kr.activations.relu(input) + kr.activations.sigmoid(input)) / 2

# Función de activación customizada que devuelve la diferencia entre softmax y softplus
def custom_activation2(input):
    return kr.activations.softmax(input) - kr.activations.softplus(input)

# Función de activación customizada que aplica la función relu al resultado de la función selu
def custom_activation3(input):
    return kr.activations.relu(kr.activations.selu(input))

---
#### Carga de datos

In [ ]:
input_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
target_data = np.array([[0], [1], [1], [0]])

#### Definición del Modelo

In [ ]:
model = kr.models.Sequential()
model.add(kr.layers.Dense(32, activation=custom_activation1))
model.add(kr.layers.Dense(16, activation=custom_activation3))
model.add(kr.layers.Dense(1, activation=custom_activation2))


#### Compilación del Modelo

In [ ]:
model.compile   (loss='mean_squared_error',
                optimizer='adam',
                metrics=['binary_accuracy'])

#### Entrenamiento del Modelo

In [ ]:
model.fit(input_data, target_data, epochs=1000)

#### Evaluación del Modelo

In [ ]:
model.evaluate(input_data, target_data)

#### Predicción del Modelo

In [ ]:
print(model.predict(input_data).round())

---
# Capa Conv 3D

Una capa convolucional es una capa que aplica un __filtro o varios__ sobre lo que se la ha dado, (véase filtros y máscaras del tema anterior). Una capa Conv3D hace esto pero en imágenes en 3 dimensiones (más una capa de canal).

Argumentos importantes:
* filters: el número de filtros.
* kernel_size: dimensiones del kernel ("máscara").
* activation: función de activación.


[Explicación Capa Conv IBM](https://developer.ibm.com/es/articles/cc-convolutional-neural-network-vision-recognition/),  
[Vídeos de DotCSV sobre Capas Convs](https://www.youtube.com/playlist?list=PL-Ogd76BhmcBaUXZGPJkmQpLgrBgGZ7v0),  
[API de Keras](https://keras.io/api/layers/convolution_layers/convolution3d/),  
[Geeks for Geeks Conv2D con todos los argumentos](https://www.geeksforgeeks.org/keras-conv2d-class/)

#### Código de preparación

Configuración del modelo

In [ ]:
# -- Preparatory code --
# Model configuration
batch_size = 100
no_epochs = 5
learning_rate = 0.005
no_classes = 10
validation_split = 0.2
verbosity = 1


Convertir vector 1D en vector 3D

In [ ]:
# Convert 1D vector into 3D values, provided by the 3D MNIST authors at
# https://www.kaggle.com/daavoo/3d-mnist
def array_to_color(array, cmap="Oranges"):
    s_m = plt.cm.ScalarMappable(cmap=cmap)
    return s_m.to_rgba(array)[:, :-1]

Reformar data a un formato que pueda manejar Conv3D

In [ ]:
# Reshape data into format that can be handled by Conv3D layers.
# Courtesy of Sam Berglin; Zheming Lian; Jiahui Jang - University of Wisconsin-Madison
# Report - https://github.com/sberglin/Projects-and-Papers/blob/master/3D%20CNN/Report.pdf
# Code - https://github.com/sberglin/Projects-and-Papers/blob/master/3D%20CNN/network_final_version.ipynb
def rgb_data_transform(data):
    data_t = []
    for i in range(data.shape[0]):
        data_t.append(array_to_color(data[i]).reshape(16, 16, 16, 3))
    return np.asarray(data_t, dtype=np.float32)

#### Código de procesamiento

Carga de datos

In [ ]:
hf = h5py.File("02_Data/full_dataset_vectors.h5", "r")


Preparación de datos

In [ ]:
# Split the data into training/test features/targets
X_train = hf["X_train"][:]
targets_train = hf["y_train"][:]
X_test = hf["X_test"][:]
targets_test = hf["y_test"][:]

# Determine sample shape
sample_shape = (16, 16, 16, 3)

# Reshape data into 3D format
X_train = rgb_data_transform(X_train)
X_test = rgb_data_transform(X_test)

# Convert target vectors to categorical targets
targets_train = kr.utils.to_categorical(targets_train).astype(np.int32)
targets_test = kr.utils.to_categorical(targets_test).astype(np.int32)

#### Creación del Modelo

In [ ]:
# Create the model
model = kr.models.Sequential()
model.add(kr.layers.Conv3D  (32, kernel_size=(3, 3, 3), activation='relu',
                            kernel_initializer='he_uniform', input_shape=sample_shape))
model.add(kr.layers.MaxPooling3D(pool_size=(2, 2, 2)))
model.add(kr.layers.Flatten())
model.add(kr.layers.Dense   (256, activation='relu',
                            kernel_initializer='he_uniform'))
model.add(kr.layers.Dense(no_classes, activation='softmax'))


#### Compilación del Modelo

In [ ]:
# Compile the model
model.compile   (loss=tensorflow.keras.losses.categorical_crossentropy,
                optimizer=tensorflow.keras.optimizers.Adam(learning_rate=learning_rate),
                metrics=['accuracy'])

#### Entrenamiento del Modelo

In [ ]:
# Fit data to model
history = model.fit (X_train, targets_train,
                    batch_size=batch_size,
                    epochs=no_epochs,
                    verbose=verbosity,
                    validation_split=validation_split)

#### Evaluación del Modelo

In [ ]:
# Generate generalization metrics
score = model.evaluate(X_test, targets_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

#### Visualización

In [ ]:
# Plot history: Categorical crossentropy & Accuracy
plt.plot(history.history['loss'],
        label='Categorical crossentropy (training data)')
plt.plot(history.history['val_loss'],
        label='Categorical crossentropy (validation data)')
plt.plot(history.history['accuracy'], label='Accuracy (training data)')
plt.plot(history.history['val_accuracy'],
        label='Accuracy (validation data)')
plt.title('Model performance for 3D MNIST Keras Conv3D example')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper right")
plt.show()